<a href="https://colab.research.google.com/github/Matrixmax/Dive_into_DeepLearning/blob/main/0x30_%E6%89%B9%E9%87%8F%E5%BD%92%E4%B8%80%E5%8C%96.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

本节我们介绍批量归一化（batch normalization）层，它能让较深的神经网络的训练变得更加容易 [1]。在3.16节（实战Kaggle比赛：预测房价）里，我们对输入数据做了标准化处理：处理后的任意一个特征在数据集中所有样本上的均值为0、标准差为1。标准化处理输入数据使各个特征的分布相近：这往往更容易训练出有效的模型。

通常来说，数据标准化预处理对于浅层模型就足够有效了。随着模型训练的进行，当每层中参数更新时，靠近输出层的输出较难出现剧烈变化。但对深层神经网络来说，即使输入数据已做标准化，训练中模型参数的更新依然很容易造成靠近输出层输出的剧烈变化。这种计算数值的不稳定性通常令我们难以训练出有效的深度模型。

批量归一化的提出正是为了应对深度模型训练的挑战。在模型训练时，批量归一化利用小批量上的均值和标准差，不断调整神经网络中间输出，从而使整个神经网络在各层的中间输出的数值更稳定。批量归一化和下一节将要介绍的残差网络为训练和设计深度模型提供了两类重要思路。

## 1. 批量归一化层
对全连接层和卷积层做批量归一化的方法稍有不同。下面我们将分别介绍这两种情况下的批量归一化。



## 1.1 对全连接层做批量归一化

对全连接层做批量的归一化

我们先考虑如何对全连接层做批量归一化。通常，我们将批量归一化层置于全连接层中的仿射变换和 激活函数之间。设全连接层的输入为 $\boldsymbol{u},$ 权重参数和偏差参数分别为 $\boldsymbol{W}$ 和 $\boldsymbol{b}$, 激活函数为 $\phi$ 。设批量 归一化的运算符为BN。那九, 使用批量归一化的全连接层的输出为
$$
\phi(\mathrm{BN}(\boldsymbol{x}))
$$
其中批量I白一化输入 $\boldsymbol{x}$ 由仿射变换
$$
x=W u+b
$$
得到。考虑一个由m个样本组成的小批量, 仿射变换的输出为一个新的小批量\mathcal{B} ~ = ~
$m,$ 批量I尸一化层的输出同样是 $d$ 维向量
$$
\boldsymbol{y}^{(i)}=\mathrm{BN}\left(\boldsymbol{x}^{(i)}\right)
$$
并由以下几步求得。首先, 对小批量\mathcal{B} 求 均 值 和 方 差 ： ~
$$
\begin{array}{c}
\boldsymbol{\mu}_{\mathcal{B}} \leftarrow \frac{1}{m} \sum_{i=1}^{m} \boldsymbol{x}^{(i)} \\
\boldsymbol{\sigma}_{\mathcal{B}}^{2} \leftarrow \frac{1}{m} \sum_{i=1}^{m}\left(\boldsymbol{x}^{(i)}-\boldsymbol{\mu}_{\mathcal{B}}\right)^{2}
\end{array}
$$
(i) 标准化:
$$
\hat{\boldsymbol{x}}^{(i)} \leftarrow \frac{\boldsymbol{x}^{(i)}-\boldsymbol{\mu}_{\mathcal{B}}}{\sqrt{\boldsymbol{\sigma}_{\mathcal{B}}^{2}+\epsilon}}
$$
这里 $\epsilon>0$ 是一个很/小的常数, 保证分母大于0。在上面标准化的基础上，批量I井一化层引入了两个可 以学习的模型参数, 拉伸 (scale) 参数 $\gamma$ 和偏移 (shift) 参数 $\beta$ 。这两个参数和 $x^{(i)}$ 形状相同, 皆为 d维向量。它们与 $\boldsymbol{x}^{(i)}$ 分别做按元素乘法 (符号\odot) 和加法计算:
\begin{array}{l}
\qquad \boldsymbol{y}^{(i)} \leftarrow \gamma \odot \hat{\boldsymbol{x}}^{(i)}+\boldsymbol{\beta} \\
\text { 至此, 我们得到了 } \boldsymbol{x}^{(i)} \text { 的批量归一化的输出 } \boldsymbol{y}^{(i)} \text { 。值得注意的是, 可学习的拉伸和偏移参数保留了不 } \\
\text { 对黑 }(i) \text { 做批量归一化的可能: 此时只需学出 } \gamma=\sqrt{\sigma_{\mathcal{B}}^{2}+\epsilon} \text { 和 } \beta=\boldsymbol{\mu}_{\mathcal{B}} \text { 。我们可以对此这样理解: 如 } \\
\text { 果批量归一化无益, 理论上, 学出的模型可以不使用批量归一化。 }
\end{array}

### 1.2 对卷积层做批量归一化

对卷积层来说，批量归一化发生在卷积计算之后、应用激活函数之前。如果卷积计算输出多个通道，我们需要对这些通道的输出分别做批量归一化，**且每个通道都拥有独立的拉伸和偏移参数，并均为标量**。设小批量中有mm个样本。在单个通道上，假设卷积计算输出的高和宽分别为pp和qq。我们需要对该通道中$m \times p \times q$个元素同时做批量归一化。对这些元素做标准化计算时，我们使用相同的均值和方差，即该通道中$m \times p \times q$个元素的均值和方差。

### 1.3 预测时的批量归一化

使用批量归一化训练时，我们可以将批量大小设得大一点，从而使批量内样本的均值和方差的计算都较为准确。将训练好的模型用于预测时，我们希望模型对于任意输入都有确定的输出。因此，单个样本的输出不应取决于批量归一化所需要的随机小批量中的均值和方差。一种常用的方法是通过移动平均估算整个训练数据集的样本均值和方差，并在预测时使用它们得到确定的输出。可见，和丢弃层一样，批量归一化层在训练模式和预测模式下的计算结果也是不一样的。

## 2.从零开始实现

In [2]:

import sys
from google.colab import drive
drive.mount('/content/drive')
sys.path.append('/content/drive/My Drive/Colab Notebooks/Dive_Into_Deep_Learning')

import time
import torch
import sys
from torch import nn,optim
import torch.nn.functional as F
import d2lzh_pytorch as d2l


Mounted at /content/drive


In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
!nvidia-smi

Wed Feb 17 09:48:49 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P8    25W / 149W |      3MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
def batch_norm(is_training,X,gamma,beta,moving_mean,moving_var,eps,momentum):
    # 判断当前模式是训练模式还是预测模式
    if not is_training:
        # 如果是在预测模式下，直接使用传入的移动平均所得的均值和方差
        X_hat = (X-moving_mean)/torch.sqrt(moving_var+eps)
    else:
        assert len(X.shape) in (2,4)
        if len(X.shape)==2:
            # 使用全连接层的情况下，计算特征维度上的均值和方差
            mean = X.mean(dim=0)
            var = ((X-mean)**2).mean(dim=0)
        else:
            # 使用二维卷积层的情况，计算通道维上（axis）的均值和方差
            # 这里我们需要保持X的形状，以方便后面可以做广播运算
            mean = X.mean(dim=0,keepdim=True).mean(dim=2,keepdim=True).mean(dim=3,keepdim=True)
            var = ((X-mean)**2).mean(dim=0,keepdim=True).mean(dim=2,keepdim=True).mean(dim=3,keepdim=True)
        # 训练模式下用当前的均值和方差做标准化
        X_hat = (X-mean)/torch.sqrt(var+eps)
        # 更新移动平均的均值和方差
        moving_mean = momentum * moving_mean +(1.0-momentum)*mean
        moving_var = momentum * moving_var + (1.0 - momentum) * var
    Y = gamma*X_hat+beta #拉伸和偏移
    return Y,moving_mean,moving_var

接下来，我们自定义一个BatchNorm层。它保存参与求梯度和迭代的拉伸参数gamma和偏移参数beta，同时也维护移动平均得到的均值和方差，以便能够在模型预测时被使用。BatchNorm实例所需指定的num_features参数对于全连接层来说应为输出个数，对于卷积层来说则为输出通道数。该实例所需指定的num_dims参数对于全连接层和卷积层来说分别为2和4。

In [19]:
class BatchNorm(nn.Module):
    def __init__(self,num_features,num_dims):
        super(BatchNorm,self).__init__()
        if num_dims==2:
            shape = (1,num_features)
        else:
            shape = (1,num_features,1,1)
        # 参与求梯度和迭代的拉伸和偏移参数，分别初始化成0和1
        self.gamma = nn.Parameter(torch.ones(shape))
        self.beta = nn.Parameter(torch.zeros(shape))
        # 不参与求梯度和迭代的变量，全在内存上初始化成0
        self.moving_mean = torch.zeros(shape)
        self.moving_var = torch.zeros(shape)
    
    def forward(self,X):
        # 如果X不在内存上，将moving_mean和moving_var复制到X所在的显存上
        if self.moving_mean.device != X.device:
            self.moving_mean = self.moving_mean.to(device)
            self.moving_var = self.moving_var.to(device)
        # 保存更新过的moving_mean和moving_var，Module实例的training属性默认为true,
        # 调用.eval()后设成false
        Y,self.moving_mean,self.moving_var = batch_norm(
            self.training,X,self.gamma,self.beta,self.moving_mean,self.moving_var,eps=1e-5,momentum=0.9
        )
        return Y

### 2.1使用批量归一化层的LeNet


In [20]:
net = nn.Sequential(
    nn.Conv2d(1,6,5),# in_channels, out_channels, kernel_size
    BatchNorm(6,num_dims=4),
    nn.Sigmoid(),
    nn.MaxPool2d(2,2),#kernel_size,stride
    nn.Conv2d(6,16,5),
    BatchNorm(16,num_dims=4),
    nn.Sigmoid(),
    nn.MaxPool2d(2,2),
    d2l.FlattenLayer(),
    nn.Linear(16*4*4,120),
    BatchNorm(120, num_dims=2),
    nn.Sigmoid(),
    nn.Linear(120, 84),
    BatchNorm(84, num_dims=2),
    nn.Sigmoid(),
    nn.Linear(84, 10)
)

In [21]:
# 训练
batch_size = 256
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size=batch_size)

lr, num_epochs = 0.001, 5
optimizer = torch.optim.Adam(net.parameters(), lr=lr)
d2l.train_ch5(net, train_iter, test_iter, batch_size, optimizer, device, num_epochs)


training on  cuda
epoch 1, loss 1.0038, train acc 0.781, test acc 0.801, time 9.6 sec
epoch 2, loss 0.4732, train acc 0.858, test acc 0.830, time 9.7 sec
epoch 3, loss 0.3794, train acc 0.874, test acc 0.860, time 9.5 sec
epoch 4, loss 0.3375, train acc 0.884, test acc 0.868, time 9.6 sec
epoch 5, loss 0.3153, train acc 0.889, test acc 0.859, time 9.6 sec


最后我们查看第一个批量归一化层学习到的拉伸参数gamma和偏移参数beta。

In [23]:
net[1].gamma.view((-1,)), net[1].beta.view((-1,))

(tensor([0.9363, 1.0371, 1.0678, 0.8997, 1.0759, 1.1215], device='cuda:0',
        grad_fn=<ViewBackward>),
 tensor([-0.5314,  0.1516,  0.4110, -0.4249, -0.5318,  0.0050], device='cuda:0',
        grad_fn=<ViewBackward>))

## 3.简洁实现

与我们刚刚自己定义的BatchNorm类相比，Pytorch中nn模块定义的BatchNorm1d和BatchNorm2d类使用起来更加简单，二者分别用于全连接层和卷积层，都需要指定输入的num_features参数值。下面我们用PyTorch实现使用批量归一化的LeNet。

In [24]:
net = nn.Sequential(
            nn.Conv2d(1, 6, 5), # in_channels, out_channels, kernel_size
            nn.BatchNorm2d(6),
            nn.Sigmoid(),
            nn.MaxPool2d(2, 2), # kernel_size, stride
            nn.Conv2d(6, 16, 5),
            nn.BatchNorm2d(16),
            nn.Sigmoid(),
            nn.MaxPool2d(2, 2),
            d2l.FlattenLayer(),
            nn.Linear(16*4*4, 120),
            nn.BatchNorm1d(120),
            nn.Sigmoid(),
            nn.Linear(120, 84),
            nn.BatchNorm1d(84),
            nn.Sigmoid(),
            nn.Linear(84, 10)
        )


In [25]:
# 使用同样的超参数进行训练。
batch_size = 256
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size=batch_size)

lr, num_epochs = 0.001, 5
optimizer = torch.optim.Adam(net.parameters(), lr=lr)
d2l.train_ch5(net, train_iter, test_iter, batch_size, optimizer, device, num_epochs)


training on  cuda
epoch 1, loss 0.9867, train acc 0.784, test acc 0.778, time 8.8 sec
epoch 2, loss 0.4518, train acc 0.866, test acc 0.749, time 9.0 sec
epoch 3, loss 0.3593, train acc 0.882, test acc 0.850, time 8.9 sec
epoch 4, loss 0.3236, train acc 0.889, test acc 0.855, time 9.1 sec
epoch 5, loss 0.3015, train acc 0.896, test acc 0.854, time 9.0 sec


## 4. 小结
1. 在模型训练时，批量归一化利用小批量上的均值和标准差，不断调整神经网络的中间输出，从而使整个神经网络在各层的中间输出的数值更稳定。
2. 对全连接层和卷积层做批量归一化的方法稍有不同。
3. 批量归一化层和丢弃层一样，在训练模式和预测模式的计算结果是不一样的。
4. PyTorch提供了BatchNorm类方便使用。